In [7]:
import pandas as pd
import json
import numpy as np

# 整合数据集

In [8]:
datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "peiyou", "ednet5w"]
datasets_dic = {"assist2009": 1, "algebra2005": 2, "bridge2algebra2006": 3, "nips_task34": 4, "ednet": 5, "peiyou": 6, "ednet5w": 7}

In [9]:
new_data = {"fold":[], "uid":[], "questions":[], "concepts":[], "responses":[], "dataset":[], "timestamps":[]}

In [10]:
for dataset in datasets:
    df_train = pd.read_csv(f"../data/{dataset}/train_valid_quelevel.csv")
    df_test = pd.read_csv(f"../data/{dataset}/test_quelevel.csv")
    df = pd.concat([df_train, df_test])
    
    data_info_ = dict()
    with open(f"../data/{dataset}/keyid2idx.json", "r") as f:
        data_info = json.load(f)
        for key in data_info:
            data_info_.setdefault(key,dict())
            try:
                for item in data_info[key]:
                    data_info_[key][data_info[key][item]] = item
            except:
                print(f"{key}")
                continue
    for i,row in df.iterrows():
        uid = data_info_["uid"][row["uid"]]
        # print(uid)
        questions = row["questions"].split(",")
        # print(f"questions:{questions}")
        concepts = row["concepts"].split(",")
        # print(f"concepts:{concepts}")
        questions = [data_info_["questions"][int(q)] for q in questions]
        new_concepts = []
        for ccc in concepts:
            ccc = ccc.split("_")
            concept = [data_info_["concepts"][int(c)] for c in ccc]
            concept = "_".join(concept)
            new_concepts.append(concept)
        new_data["fold"].append(row["fold"])
        new_data["uid"].append(uid)
        new_data["questions"].append(",".join(questions))
        new_data["concepts"].append(",".join(new_concepts))
        new_data["responses"].append(row["responses"])
        new_data["dataset"].append(dataset)
        if "timestamps" in row:
            new_data["timestamps"].append(row["timestamps"])
        else:
            new_data["timestamps"].append(",".join([str(i) for i in range(len(questions))]))

max_concepts
max_concepts
max_concepts
max_concepts
max_concepts
max_concepts
max_concepts


In [11]:
new_df = pd.DataFrame(new_data)

In [12]:
new_df.to_csv(f"../data/pretrain/train_valid_quelevel_pretrain.csv", index=None)

In [13]:
new_df[:5]

,fold,uid,questions,concepts,responses,dataset,timestamps
0,0,82652,"87531,87613,87501,87479,87509,87543,88116,8801...","311,311,311,311,311,311,311,311,311,311,311,31...","0,0,0,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1",assist2009,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18"
1,0,88651,"86532,86552,53785,53797,53782,53794,86348,8635...","280,280,280,280,280,280,280,280,280","0,0,1,1,1,1,0,0,1",assist2009,"0,1,2,3,4,5,6,7,8"
2,0,91795,"119374,119248,119309,119245,119368,119298,119288","18,18,18,18,18,18,18","1,0,1,1,1,1,1",assist2009,"0,1,2,3,4,5,6"
3,0,87261,"71345,71361,71340,71349,87173,87162,87178,8718...","70,70,70,70,81,81,81,81,81,81,81","0,1,1,0,0,1,1,0,1,0,1",assist2009,"0,1,2,3,4,5,6,7,8,9,10"
4,0,89951,"119252,119375,119358,119288,119289,119290,1192...","18,18,18,18,18,18,18,18,18,18,18,18,18,17","0,0,1,0,0,0,0,1,1,1,1,1,1,0",assist2009,"0,1,2,3,4,5,6,7,8,9,10,11,12,13"


In [14]:
df_sort = pd.read_csv('../data/pretrain/train_valid_quelevel_pretrain.csv')
df_sort = df_sort.sort_values(by='uid',ascending = True)

In [15]:
df_sort[:5]

,fold,uid,questions,concepts,responses,dataset,timestamps
16100,0,0,"355,1545,827,316,1638,1530,368,658,1743,3186,3...","2390,1291_2235,1377,1377,108,1377,2235,295,291...","1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",peiyou,"1599481857000,1599481857000,1599481857000,1599..."
4014,1,02ZjVTxC34,"LDEMO####WKST----R2C1,LDEMO####WKST----R2C2,LD...","Identifying units,Identifying units,Entering a...","1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,...",algebra2005,"1125982844000,1125982906000,1125982947000,1125..."
4148,3,02i5jCrfQK,"LDEMO####WKST----R2C1,LDEMO####WKST----R2C2,LD...","Identifying units,Identifying units,Entering a...","1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,0,1,1,...",algebra2005,"1125378230000,1125378243000,1125378254000,1125..."
4235,4,0493L6bGa9,"LIT60A----q*c*(v+r) = d*x,LIT60A----q*c*(v+r)/...",[SkillRule: Remove coefficient; {ax+b=c@@@@ di...,"0,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,...",algebra2005,"1140486334000,1140486457000,1140486942000,1140..."
3892,0,04N6FdP4pd,"L5FB16----R2C1,L5FB16----R3C1,L5FB16----R2C2,L...","Identifying units,Define Variable,Identifying ...","1,1,1,0,0,0,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,1,0,...",algebra2005,"1140654945000,1140654947000,1140654967000,1140..."


# 完成数据映射

In [16]:
def id_mapping_que(df):
    id_keys = ["questions", "concepts", "uid"]
    dres = dict()
    dkeyid2idx = dict()
    # print(f"df.columns: {df.columns}")
    flag = True #判定数据集有没有变化
    pre_data = "assist2009"
    for key in df.columns:
        if key not in id_keys:
            dres[key] = df[key]
    for i, row in df.iterrows():
        dataset = row["dataset"]
        if dataset != "ednet5w":
            for j,key in enumerate(id_keys):
                if key not in df.columns:
                    continue
                if key == "timestamps":
                    dres[key].append(row[key])
                else:
                    dkeyid2idx.setdefault(key, dict())
                    dkeyid2idx[key].setdefault(dataset, dict())
                    dres.setdefault(key, [])
                    curids = []
                    # print(f"dkeyid2idx:{dkeyid2idx}")
                    for id in row[key].split(","):
                        sub_ids = id.split('_')
                        sub_curids = []
                        for sub_id in sub_ids:
                            if sub_id not in dkeyid2idx[key][dataset]:
                                if list(dkeyid2idx[key][dataset].values()) == []: #到新的数据集
                                    if dataset == "assist2009":
                                        dkeyid2idx[key][dataset][sub_id] = 0
                                        flag = False
                                    else:
                                        # print(f"key:{key}, dataset:{dataset}, pre_data:{pre_data}")
                                        # print(f"error:{dkeyid2idx[key][pre_data]}")
                                        dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][pre_data].values())[-1]+1
                                else: #原本数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][dataset].values())[-1]+1
                                # dkeyid2idx[key][dataset][sub_id] = cnt+1
                            sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            # cnt += 1
                        curids.append("_".join(sub_curids))
                    dres[key].append(",".join(curids))
            pre_data = dataset
        else:
            for j,key in enumerate(id_keys):
                if key not in df.columns:
                    continue
                if key == "timestamps":
                    dres[key].append(row[key])
                else:
                    dkeyid2idx.setdefault(key, dict())
                    dkeyid2idx[key].setdefault(dataset, dict())
                    dres.setdefault(key, [])
                    curids = []
                    # print(f"dkeyid2idx:{dkeyid2idx}")
                    for id in row[key].split(","):
                        sub_ids = id.split('_')
                        sub_curids = []
                        for sub_id in sub_ids:
                            if sub_id in dkeyid2idx[key]["ednet"]:
                                sub_curids.append(str(dkeyid2idx[key]["ednet"][sub_id]))
                            elif sub_id not in dkeyid2idx[key]["ednet"] and sub_id not in dkeyid2idx[key][dataset]:
                                if list(dkeyid2idx[key][dataset].values()) == []: #到新的数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key]["ednet"].values())[-1]+1
                                else: #原本数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][dataset].values())[-1]+1
                                sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            elif sub_id in dkeyid2idx[key][dataset]:
                                sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            # cnt += 1
                        curids.append("_".join(sub_curids))
                    dres[key].append(",".join(curids))
    finaldf = pd.DataFrame(dres)
    return finaldf, dkeyid2idx

In [17]:
finaldf, dkeyid2idx = id_mapping_que (new_df)

In [18]:
with open(f"../data/pretrain/keyid2idx.json", "w") as f:
    json.dump(dkeyid2idx, f)

In [19]:
list(dkeyid2idx['questions']['ednet'].values())[-1]

332961

In [20]:
len(list(dkeyid2idx['questions']['ednet5w'].values()))

339

In [21]:
for i, row in enumerate(finaldf["questions"]):
    row_ = row.split(",")
    try:
        
        row_ = [int(x) for x in row_]
    except:
        print(i,row)
        break

In [22]:
len(dkeyid2idx['questions']['bridge2algebra2006'])

129263

In [23]:
len(dkeyid2idx['questions']['algebra2005'])

173113

In [24]:
list(dkeyid2idx['questions']['ednet5w'].values())[-1]

333300

In [25]:
finaldf.head(5)

,fold,responses,dataset,timestamps,questions,concepts,uid
0,0,"0,0,0,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1",assist2009,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18","0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1",0
1,0,"0,0,1,1,1,1,0,0,1",assist2009,"0,1,2,3,4,5,6,7,8","19,20,21,22,23,24,25,26,27","2,2,2,2,2,2,2,2,2",1
2,0,"1,0,1,1,1,1,1",assist2009,"0,1,2,3,4,5,6","28,29,30,31,32,33,34","3,3,3,3,3,3,3",2
3,0,"0,1,1,0,0,1,1,0,1,0,1",assist2009,"0,1,2,3,4,5,6,7,8,9,10","35,36,37,38,39,40,41,42,43,44,45","4,4,4,4,5,5,5,5,5,5,5",3
4,0,"0,0,1,0,0,0,0,1,1,1,1,1,1,0",assist2009,"0,1,2,3,4,5,6,7,8,9,10,11,12,13","46,47,48,34,49,50,51,52,53,54,55,56,57,58","3,3,3,3,3,3,3,3,3,3,3,3,3,6",4


# 计算一题对应最多知识点数量

In [26]:
def get_max_concepts(df):
    max_concepts = 1
    for i, row in df.iterrows():
        cs = row["concepts"].split(",")
        num_concepts = max([len(c.split("_")) for c in cs])
        if num_concepts >= max_concepts:
            max_concepts = num_concepts
    return max_concepts

In [27]:
max_concepts = get_max_concepts(new_df)

In [28]:
max_concepts

7

# 数据统计

In [29]:
def calStatistics(df, stares, key):
    allin, allselect = 0, 0
    allqs, allcs = set(), set()
    for i, row in df.iterrows():
        rs = row["responses"].split(",")
        curlen = len(rs) - rs.count("-1")
        allin += curlen
        if "selectmasks" in row:
            ss = row["selectmasks"].split(",")
            slen = ss.count("1")
            allselect += slen
        if "concepts" in row:
            cs = row["concepts"].split(",")
            fc = list()
            for c in cs:
                cc = c.split("_")
                fc.extend(cc)
            curcs = set(fc) - {"-1"}
            allcs |= curcs
        if "questions" in row:
            qs = row["questions"].split(",")
            curqs = set(qs) - {"-1"}
            allqs |= curqs
    stares.append(",".join([str(s) for s in [key, allin, df.shape[0], allselect]]))
    return allin, allselect, len(allqs), len(allcs), df.shape[0]

In [30]:
finaldf[finaldf.dataset=="ednet"]

,fold,responses,dataset,timestamps,questions,concepts,uid
10489,0,"1,0,0,1,1,0,0,0,0,0,0,1",ednet,"1563610453940,1563610478874,1563610525266,1563...","321061,321062,321063,321064,321065,321066,3210...","785,786,787,788,789,790,791,792_793_794_795_79...",10489
10490,0,"1,1,1,0,1,1,1",ednet,"1549710575907,1549710597976,1549710631782,1549...","321073,321074,321075,321076,321077,321078,321079","808,787,804_802_803_807_799,787,809,810,793_79...",10490
10491,0,"1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,1",ednet,"1552497596582,1555290666039,1555290700445,1555...","321080,321081,321082,321083,321084,321085,3210...","811,787,787,791,809,812,811,813,814_815_801_80...",10491
10492,0,"0,1,0,0,0,0,1",ednet,"1556680865929,1556680889054,1556680913845,1556...","321096,321097,321098,321099,321100,321101,321102","812,821,812,822,823,824,825",10492
10493,0,"0,1,1,1,0,0,0,1,1,0,0,1,0,0,1,1,0,1,1,0,1,1,1,...",ednet,"1567870303805,1567870329459,1567870350848,1567...","321065,321103,321104,321105,321078,321106,3210...","789,826_815_795,788,827,810,791,809,790,828,81...",10493
...,...,...,...,...,...,...,...
15171,-1,"1,0,1,1,1,0,0,0,1,1,1,0,1,0,1,0,0,1,1,0,0,0,1,...",ednet,"1523786634547,1523786664134,1523786695449,1523...","321188,321189,321190,321191,321192,321194,3211...","862_815_795,863_864_800,865_864_800,817_837_79...",15171
15172,-1,"1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,...",ednet,"1564527136463,1564527162902,1564527194030,1564...","329435,322191,321158,321063,321105,321109,3210...","787,786,852,787,827,812,809,897_864_795,894_89...",15172
15173,-1,"1,0,0,1,1,0,1",ednet,"1546121385503,1546121407944,1546121430431,1546...","321421,327467,321073,332086,322866,328646,321109","787,791,808,787,806_798_800_796,921,812",15173
15174,-1,"1,1,0,0,1,1,0,1,1,0,0,1,1,0",ednet,"1574029122000,1574029146000,1574029157000,1574...","329418,321163,324900,321460,326536,324261,3241...","832,817_820_807_800,809,847,811,874,791,832,81...",15174


In [31]:
ins, ss, qs, cs, seqnum = calStatistics(df=finaldf, stares=[], key="original train+valid question level")

In [32]:
ins, ss, qs, cs, seqnum

(16168012, 0, 340614, 1838, 80096)

In [33]:
for data in datasets:
    final_sub_df = finaldf[finaldf.dataset==data]
    ins_, ss_, qs_, cs_, seqnum_ = calStatistics(df=final_sub_df, stares=[], key="original train+valid question level")
    print(f"dataset:{data}, ins_:{ins_}, ss_:{ss_}, qs_:{qs_}, cs_:{cs_}, seqnum_:{seqnum_}")

dataset:assist2009, ins_:282619, ss_:0, qs_:17737, cs_:123, seqnum_:3852
dataset:algebra2005, ins_:607025, ss_:0, qs_:173113, cs_:112, seqnum_:574
dataset:bridge2algebra2006, ins_:1817474, ss_:0, qs_:129263, cs_:493, seqnum_:1145
dataset:nips_task34, ins_:1382727, ss_:0, qs_:948, cs_:57, seqnum_:4918
dataset:ednet, ins_:596600, ss_:0, qs_:11901, cs_:188, seqnum_:4687
dataset:peiyou, ins_:5549635, ss_:0, qs_:7652, cs_:865, seqnum_:18066
dataset:ednet5w, ins_:5931932, ss_:0, qs_:12235, cs_:188, seqnum_:46854


# 把数据集划分成sequence

In [34]:

def generate_sequences(df, effective_keys, min_seq_len=3, maxlen = 200, pad_val = -1):
    save_keys = list(effective_keys) + ["selectmasks"]
    dres = {"selectmasks": []}

    dropnum = 0
    for i, row in df.iterrows():
        dcur = save_dcur(row, effective_keys)

        rest, lenrs = len(dcur["responses"]), len(dcur["responses"])
        j = 0
        while lenrs >= j + maxlen:   
            rest = rest - (maxlen)
            for key in effective_keys:
                dres.setdefault(key, [])
                if key not in ONE_KEYS:
                    dres[key].append(",".join(dcur[key][j: j + maxlen]))#[str(k) for k in dcur[key][j: j + maxlen]]))
                else:
                    dres[key].append(dcur[key])
            dres["selectmasks"].append(",".join(["1"] * maxlen))

            j += maxlen
        if rest < min_seq_len:# delete sequence len less than min_seq_len
            dropnum += rest
            continue
        
        pad_dim = maxlen - rest
        for key in effective_keys:
            dres.setdefault(key, [])
            if key not in ONE_KEYS:
                paded_info = np.concatenate([dcur[key][j:], np.array([pad_val] * pad_dim)])
                dres[key].append(",".join([str(k) for k in paded_info]))
            else:
                if key == "dataset":
                    dres[key].append(datasets_dic[dcur[key]])
                else:
                    dres[key].append(dcur[key])
        dres["selectmasks"].append(",".join(["1"] * rest + [str(pad_val)] * pad_dim))
    
    # after preprocess data, report
    dfinal = dict()
    for key in ALL_KEYS:
        if key in save_keys:
            dfinal[key] = dres[key]
    finaldf = pd.DataFrame(dfinal)
    print(f"dropnum: {dropnum}")
    return finaldf

In [35]:
def save_dcur(row, effective_keys):
    dcur = dict()
    for key in effective_keys:
        if key not in ONE_KEYS:
            dcur[key] = row[key].split(",")#[int(i) for i in row[key].split(",")]
        else:
            dcur[key] = row[key]
    return dcur

In [36]:
ONE_KEYS = ["fold", "uid","dataset"]

In [37]:
ALL_KEYS = ["fold", "uid", "dataset", "questions", "concepts", "responses", "timestamps", "usetimes", "selectmasks", "is_repeat", "qidxs", "rest", "orirow","cidxs"]

In [38]:
split_seqs = generate_sequences(df=finaldf, effective_keys={"uid", "dataset", "questions","concepts","responses","fold", "timestamps"}, min_seq_len=3, maxlen=1024)

dropnum: 8


In [39]:
ins, ss, qs, cs, seqnum = calStatistics(df=split_seqs, stares=[], key="train+valid sequences question level")

In [40]:
ins, ss, qs, cs, seqnum

(16168004, 16168004, 340614, 1838, 84556)

In [41]:
ins, ss, qs, cs, seqnum

(16168004, 16168004, 340614, 1838, 84556)

In [42]:
split_seqs.shape

(84556, 8)

In [43]:
finaldf.shape

(80096, 7)

In [44]:
split_seqs

,fold,uid,dataset,questions,concepts,responses,timestamps,selectmasks
0,0,0,1,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,-1,-1,-1...","0,0,0,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,-1,-1,-1...","0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1..."
1,0,1,1,"19,20,21,22,23,24,25,26,27,-1,-1,-1,-1,-1,-1,-...","2,2,2,2,2,2,2,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","0,0,1,1,1,1,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","0,1,2,3,4,5,6,7,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-..."
2,0,2,1,"28,29,30,31,32,33,34,-1,-1,-1,-1,-1,-1,-1,-1,-...","3,3,3,3,3,3,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","1,0,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","0,1,2,3,4,5,6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1..."
3,0,3,1,"35,36,37,38,39,40,41,42,43,44,45,-1,-1,-1,-1,-...","4,4,4,4,5,5,5,5,5,5,5,-1,-1,-1,-1,-1,-1,-1,-1,...","0,1,1,0,0,1,1,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,...","0,1,2,3,4,5,6,7,8,9,10,-1,-1,-1,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,..."
4,0,4,1,"46,47,48,34,49,50,51,52,53,54,55,56,57,58,-1,-...","3,3,3,3,3,3,3,3,3,3,3,3,3,6,-1,-1,-1,-1,-1,-1,...","0,0,1,0,0,0,0,1,1,1,1,1,1,0,-1,-1,-1,-1,-1,-1,...","0,1,2,3,4,5,6,7,8,9,10,11,12,13,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,..."
...,...,...,...,...,...,...,...,...
84551,-1,62027,ednet5w,"321188,321189,321190,321191,321194,321193,3211...","862_815_795,863_864_800,865_864_800,817_837_79...","1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,1,0,1,...","1534310781888,1534311287783,1534311320682,1534...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84552,-1,62027,ednet5w,"322227,331585,330120,327279,325549,321739,3322...","829,813,811,884,889,917,791,917,786,860,811,78...","1,1,1,0,0,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,1,...","1535555585290,1535555683148,1535555753463,1535...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84553,-1,62027,7,"321662,321509,331697,322433,324584,328927,3281...","883,844,809,823,786,833,823,790,825,790,823,82...","0,1,1,1,0,0,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,1,0,...","1541505283370,1541505306604,1541505386208,1541...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84554,-1,62028,7,"327296,331772,321065,323211,323725,-1,-1,-1,-1...","825,791,789,841_842_799_807,804_817_850_807_80...","0,1,0,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...","1541725068973,1541725100323,1541725132987,1541...","1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,..."


In [45]:
test_seqs = split_seqs
test_seqs

,fold,uid,dataset,questions,concepts,responses,timestamps,selectmasks
0,0,0,1,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,-1,-1,-1...","0,0,0,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,-1,-1,-1...","0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1..."
1,0,1,1,"19,20,21,22,23,24,25,26,27,-1,-1,-1,-1,-1,-1,-...","2,2,2,2,2,2,2,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","0,0,1,1,1,1,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","0,1,2,3,4,5,6,7,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-..."
2,0,2,1,"28,29,30,31,32,33,34,-1,-1,-1,-1,-1,-1,-1,-1,-...","3,3,3,3,3,3,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","1,0,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","0,1,2,3,4,5,6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1..."
3,0,3,1,"35,36,37,38,39,40,41,42,43,44,45,-1,-1,-1,-1,-...","4,4,4,4,5,5,5,5,5,5,5,-1,-1,-1,-1,-1,-1,-1,-1,...","0,1,1,0,0,1,1,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,...","0,1,2,3,4,5,6,7,8,9,10,-1,-1,-1,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,..."
4,0,4,1,"46,47,48,34,49,50,51,52,53,54,55,56,57,58,-1,-...","3,3,3,3,3,3,3,3,3,3,3,3,3,6,-1,-1,-1,-1,-1,-1,...","0,0,1,0,0,0,0,1,1,1,1,1,1,0,-1,-1,-1,-1,-1,-1,...","0,1,2,3,4,5,6,7,8,9,10,11,12,13,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,..."
...,...,...,...,...,...,...,...,...
84551,-1,62027,ednet5w,"321188,321189,321190,321191,321194,321193,3211...","862_815_795,863_864_800,865_864_800,817_837_79...","1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,1,0,1,...","1534310781888,1534311287783,1534311320682,1534...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84552,-1,62027,ednet5w,"322227,331585,330120,327279,325549,321739,3322...","829,813,811,884,889,917,791,917,786,860,811,78...","1,1,1,0,0,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,1,...","1535555585290,1535555683148,1535555753463,1535...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84553,-1,62027,7,"321662,321509,331697,322433,324584,328927,3281...","883,844,809,823,786,833,823,790,825,790,823,82...","0,1,1,1,0,0,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,1,0,...","1541505283370,1541505306604,1541505386208,1541...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84554,-1,62028,7,"327296,331772,321065,323211,323725,-1,-1,-1,-1...","825,791,789,841_842_799_807,804_817_850_807_80...","0,1,0,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...","1541725068973,1541725100323,1541725132987,1541...","1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,..."


In [46]:
test_seqs['dataset'] = test_seqs['dataset'].apply(lambda x: datasets_dic[x] if x in datasets_dic else x)

In [47]:
test_seqs

,fold,uid,dataset,questions,concepts,responses,timestamps,selectmasks
0,0,0,1,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,-1,-1,-1...","0,0,0,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,-1,-1,-1...","0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1..."
1,0,1,1,"19,20,21,22,23,24,25,26,27,-1,-1,-1,-1,-1,-1,-...","2,2,2,2,2,2,2,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","0,0,1,1,1,1,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","0,1,2,3,4,5,6,7,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-...","1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-..."
2,0,2,1,"28,29,30,31,32,33,34,-1,-1,-1,-1,-1,-1,-1,-1,-...","3,3,3,3,3,3,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","1,0,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","0,1,2,3,4,5,6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1..."
3,0,3,1,"35,36,37,38,39,40,41,42,43,44,45,-1,-1,-1,-1,-...","4,4,4,4,5,5,5,5,5,5,5,-1,-1,-1,-1,-1,-1,-1,-1,...","0,1,1,0,0,1,1,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,...","0,1,2,3,4,5,6,7,8,9,10,-1,-1,-1,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,..."
4,0,4,1,"46,47,48,34,49,50,51,52,53,54,55,56,57,58,-1,-...","3,3,3,3,3,3,3,3,3,3,3,3,3,6,-1,-1,-1,-1,-1,-1,...","0,0,1,0,0,0,0,1,1,1,1,1,1,0,-1,-1,-1,-1,-1,-1,...","0,1,2,3,4,5,6,7,8,9,10,11,12,13,-1,-1,-1,-1,-1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,..."
...,...,...,...,...,...,...,...,...
84551,-1,62027,7,"321188,321189,321190,321191,321194,321193,3211...","862_815_795,863_864_800,865_864_800,817_837_79...","1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,1,0,1,...","1534310781888,1534311287783,1534311320682,1534...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84552,-1,62027,7,"322227,331585,330120,327279,325549,321739,3322...","829,813,811,884,889,917,791,917,786,860,811,78...","1,1,1,0,0,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,1,...","1535555585290,1535555683148,1535555753463,1535...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84553,-1,62027,7,"321662,321509,331697,322433,324584,328927,3281...","883,844,809,823,786,833,823,790,825,790,823,82...","0,1,1,1,0,0,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,1,0,...","1541505283370,1541505306604,1541505386208,1541...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
84554,-1,62028,7,"327296,331772,321065,323211,323725,-1,-1,-1,-1...","825,791,789,841_842_799_807,804_817_850_807_80...","0,1,0,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...","1541725068973,1541725100323,1541725132987,1541...","1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,..."


In [48]:
test_seqs.to_csv(f"../data/pretrain/train_valid_sequences_quelevel_1024.csv", index=None)

In [49]:
finaldf.to_csv(f"../data/pretrain/train_valid_quelevel.csv", index=None)

# 完成测试集映射

In [59]:
datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "ednet5w", "peiyou"]

In [60]:
def map_dataset(datasets):
    new_data = {"fold":[], "uid":[], "questions":[], "concepts":[], "responses":[], "dataset":[], "timestamps":[]}
    for dataset in datasets:
        datapath = f"../data/{dataset}/test_quelevel.csv"
        df = pd.read_csv(datapath)
        data_info_ = dict()
        with open(f"../data/{dataset}/keyid2idx.json") as f:
            data_info = json.load(f)
            for key in data_info:
                data_info_.setdefault(key,dict())
                try:
                    for item in data_info[key]:
                        data_info_[key][data_info[key][item]] = item
                except:
                    print(f"{key}")
                    continue
        for i,row in df.iterrows():
            uid = data_info_["uid"][row["uid"]]
            questions = row["questions"].split(",")
            # print(f"questions:{questions}")
            concepts = row["concepts"].split(",")
            # print(f"concepts:{concepts}")
            questions = [data_info_["questions"][int(q)] for q in questions]
            new_concepts = []
            for ccc in concepts:
                ccc = ccc.split("_")
                concept = [data_info_["concepts"][int(c)] for c in ccc]
                concept = "_".join(concept)
                new_concepts.append(concept)
            new_data["fold"].append(row["fold"])
            new_data["uid"].append(uid)
            new_data["questions"].append(",".join(questions))
            new_data["concepts"].append(",".join(new_concepts))
            new_data["responses"].append(row["responses"])
            new_data["dataset"].append(dataset)
            if "timestamps" in row:
                new_data["timestamps"].append(row["timestamps"])
            else:
                new_data["timestamps"].append(",".join([str(i) for i in range(len(questions))]))
    return new_data

In [61]:
new_data = map_dataset(datasets)

max_concepts
max_concepts
max_concepts
max_concepts
max_concepts
max_concepts
max_concepts


In [62]:
new_df = pd.DataFrame(new_data)

In [63]:
new_df[:5]

,fold,uid,questions,concepts,responses,dataset,timestamps
0,-1,77949,"93375,93448,93394,93453,93467,93380,93411,8594...","2_37_70,2_37_48_77,2_37_70,2_37_48_77,2_37_48_...","0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0",assist2009,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15"
1,-1,82929,"66079,52813,66485,97860","49_50,49,49,50","1,1,1,1",assist2009,"0,1,2,3"
2,-1,78575,"84985,84635,84614,84595,84602,84615,84591,8623...","309,310,309,309,309,309,309,65,65,65,65,65,50,...","0,1,1,0,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,...",assist2009,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18..."
3,-1,79833,"51010,51009,50992,50987,85716,85693,85750,8569...","310,310,310,310,49,49,49,49,50,50,50,50,50,49_...","0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0,0,...",assist2009,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18..."
4,-1,83292,"53265,53313,53330,53321,53319,53299,66701,6663...","51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,2...","0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,0,0,1,1,1,1,1,...",assist2009,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18..."


In [64]:
with open(f"../data/pretrain/keyid2idx.json","r") as f:
    dkeyid2idx = json.load(f)

## 除ednet5w

In [ ]:
for dataset in datasets:
    if dataset not in ["ednet5w"]:
        sub_df = new_df[new_df.dataset==dataset]
        new_map_data = {"fold":[], "uid":[], "questions":[], "concepts":[], "responses":[], "timestamps":[]}
        for i,row in sub_df.iterrows():
            questions = [str(dkeyid2idx["questions"][dataset][x]) for x in row["questions"].split(",")]
            concepts = row["concepts"].split(",")
            new_concepts = []
            for ccc in concepts:
                ccc = ccc.split("_")
                concept = [str(dkeyid2idx["concepts"][dataset][c]) for c in ccc]
                # concept = [str(dkeyid2idx.get("concepts").get(dataset).get(c,len(dkeyid2idx['concepts'][dataset]))) for c in ccc]
                concept = "_".join(concept)
                new_concepts.append(concept)
            new_map_data["fold"].append(row["fold"])
            new_map_data["uid"].append(row["uid"])
            new_map_data["questions"].append(",".join(questions))
            new_map_data["concepts"].append(",".join(new_concepts))
            new_map_data["responses"].append(row["responses"])
            new_map_data["timestamps"].append(row["timestamps"])
            new_map_df = pd.DataFrame(new_map_data)
            new_map_df.to_csv(f"{uni_path}/{dataset}/test_quelevel_pretrain.csv")

In [65]:
datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "peiyou"]
for dataset in datasets:
    sub_df = new_df[new_df.dataset==dataset]
    new_map_data = {"fold":[], "uid":[], "questions":[], "concepts":[], "responses":[], "timestamps":[]}
    for i,row in sub_df.iterrows():
        questions = [str(dkeyid2idx["questions"][dataset][x]) for x in row["questions"].split(",")]
        concepts = row["concepts"].split(",")
        dataset = datasets_dic[dataset]
        new_concepts = []
        for ccc in concepts:
            ccc = ccc.split("_")
            concept = [str(dkeyid2idx["concepts"][dataset][c]) for c in ccc]
            # concept = [str(dkeyid2idx.get("concepts").get(dataset).get(c,len(dkeyid2idx['concepts'][dataset]))) for c in ccc]
            concept = "_".join(concept)
            new_concepts.append(concept)
        new_map_data["fold"].append(row["fold"])
        new_map_data["uid"].append(row["uid"])
        new_map_data["questions"].append(",".join(questions))
        new_map_data["concepts"].append(",".join(new_concepts))
        new_map_data["responses"].append(row["responses"])
        new_map_data["timestamps"].append(row["timestamps"])
        new_map_df = pd.DataFrame(new_map_data)
        new_map_df.to_csv(f"../data/{dataset}/test_quelevel_pretrain.csv")

KeyError: 1

In [ ]:
datasets = ["ednet5w"]
for dataset in datasets:
    sub_df = new_df[new_df.dataset==dataset]
    new_map_data = {"fold":[], "uid":[], "questions":[], "concepts":[], "responses":[], "timestamps":[]}
    for i,row in sub_df.iterrows():
        questions = []
        for x in row["questions"].split(","):
            if x in dkeyid2idx["questions"]["ednet"]:
                questions.append(str(dkeyid2idx["questions"]["ednet"][x]))
            else:
                questions.append(str(dkeyid2idx["questions"][dataset][x]))
        
        concept = []
        concepts = row["concepts"].split(",")
        new_concepts = []
        for ccc in concepts:
            ccc = ccc.split("_")
            concept = []
            for c in ccc:
                if c in dkeyid2idx['concepts']["ednet"]:
                    concept.append(str(dkeyid2idx["concepts"]["ednet"][c]))
                else:
                    concept.append(str(dkeyid2idx["concepts"][dataset][c]))
            concept = "_".join(concept)
            new_concepts.append(concept)
        new_map_data["fold"].append(row["fold"])
        new_map_data["uid"].append(row["uid"])
        new_map_data["questions"].append(",".join(questions))
        new_map_data["concepts"].append(",".join(new_concepts))
        new_map_data["responses"].append(row["responses"])
        new_map_data["timestamps"].append(row["timestamps"])
        new_map_df = pd.DataFrame(new_map_data)
        new_map_df.to_csv(f"../data/{dataset}/test_quelevel_pretrain.csv")

KeyboardInterrupt: 

In [ ]:
new_map_df

## 生成window测试集

In [ ]:
def generate_window_sequences(df, effective_keys, maxlen=200, pad_val=-1):
    save_keys = list(effective_keys) + ["selectmasks"]
    dres = {"selectmasks": []}
    for i, row in df.iterrows():
        dcur = save_dcur(row, effective_keys)
        lenrs = len(dcur["responses"])
        if lenrs > maxlen:
            for key in effective_keys:
                dres.setdefault(key, [])
                if key not in ONE_KEYS:
                    dres[key].append(",".join(dcur[key][0: maxlen]))#[str(k) for k in dcur[key][0: maxlen]]))
                else:
                    dres[key].append(dcur[key])
            dres["selectmasks"].append(",".join(["1"] * maxlen))
            for j in range(maxlen+1, lenrs+1):
                for key in effective_keys:
                    dres.setdefault(key, [])
                    if key not in ONE_KEYS:
                        dres[key].append(",".join([str(k) for k in dcur[key][j-maxlen: j]]))
                    else:
                        dres[key].append(dcur[key])
                dres["selectmasks"].append(",".join([str(pad_val)] * (maxlen - 1) + ["1"]))
        else:
            for key in effective_keys:
                dres.setdefault(key, [])
                if key not in ONE_KEYS:
                    pad_dim = maxlen - lenrs
                    paded_info = np.concatenate([dcur[key][0:], np.array([pad_val] * pad_dim)])
                    dres[key].append(",".join([str(k) for k in paded_info]))
                else:
                    dres[key].append(dcur[key])
            dres["selectmasks"].append(",".join(["1"] * lenrs + [str(pad_val)] * pad_dim))
    
    dfinal = dict()
    for key in ALL_KEYS:
        if key in save_keys:
            # print(f"key: {key}, len: {len(dres[key])}")
            dfinal[key] = dres[key]
    finaldf = pd.DataFrame(dfinal)
    return finaldf

In [ ]:
datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "peiyou", "ednet5w"]
for data in datasets:
    test_df = pd.read_csv(f"/root/autodl-nas/project/pykt_nips2022/data/{data}/test_quelevel_pretrain.csv")
    test_window_seqs = generate_window_sequences(test_df, list({"uid","questions","concepts","responses","fold"}), maxlen=200)
    test_window_seqs.to_csv(f"/root/autodl-nas/project/pykt_nips2022/data/{data}/test_window_sequences_quelevel_pretrain_200.csv")

In [ ]:
test_df[:5]

In [ ]:
test_window_seqs = generate_window_sequences(test_df, list({"uid","questions","concepts","responses","fold"}), maxlen=1024)

In [ ]:
test_window_seqs

In [ ]:
import matplotlib.pyplot as plt

# 你的数据
data = [229471, 513430, 1463212, 1109911, 462626, 4719979, 4446825]

# 数据标签
labels = ['229471', '513430', '1463212', '1109911', '462626', '4719979', '4446825']

# 创建饼图
plt.pie(data, labels = labels, autopct='%1.1f%%')

# 添加标题
plt.title('数据饼图')

# 显示图表
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 你的数据
data = [229471, 513430, 1463212, 1109911, 462626, 4719979, 4446825]

# 数据标签
labels = ["AS2009", "AL2005", "BD2006", "NIPS34", "EdNet", "EdNet5w", "Peiyou"]

# 创建饼图
plt.pie(data, labels = labels, autopct='%1.1f%%')

# 添加标题
plt.title('Pretrain data distribution')

plt.savefig("pie_chart.pdf", format='pdf')

# 显示图表
plt.show()


In [ ]:
path = "/root/autodl-nas/project/pykt_nips2022/examples/best_model_path/ednet5w/"

In [ ]:
import os

In [ ]:
models = os.listdir(path)

In [ ]:
for model in models:
    model_path = os.listdir(os.path.join(path, model))
    for best_path in model_path:
        print("- "+os.path.join(path, model, best_path))